In [1]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt

from qiskit import Aer
from tqdm.notebook import tqdm
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

import sys
sys.path.insert(0, '../../src/')
from neuralnetwork import *
from analysis import *

#%matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
class QuantumKernel:
    def __init__(self, n_qubits, reps, shots):
        self.n_qubits = n_qubits
        self.shots = shots
        self.reps = reps #number of times to repeat the ansatz
        
        self.n_params = self.reps*self.n_qubits + 1
        self.params = np.random.uniform(-np.pi, np.pi, self.n_params)
        
        
    def encoder(self, circuit, storage, x):
        """Encodes the data set to a quantum state"""
        for i, _x in enumerate(x):
            circuit.rx(_x, storage[i])
        
        return circuit
        
        
    def ansatz(self, circuit, storage, params):
        """Encodes the parameters and adds entanglement to transform the state"""
        for i, param in enumerate(params):
            circuit.ry(param, storage[i])
            
        for i in range(self.n_qubits-1):
            circuit.cx(storage[i], storage[i+1])
            
        return circuit
    
    
    def ansatz_bias(self, circuit, storage, param):
        """One last rotation on the last qubit that is to be measured"""
        circuit.ry(param, storage[-1])
        
        return circuit
    
    
    def evaluate(self, data):
        outputs = []
        for x in data:
            storage = qk.QuantumRegister(self.n_qubits)
            clas_reg = qk.ClassicalRegister(1)
            circuit = qk.QuantumCircuit(storage, clas_reg)
            
            #encode the data to the quantum state
            circuit = self.encoder(circuit, storage, x)
            
            #indicies for parameter slices
            idx_start = 0
            idx_end = self.n_qubits
            
            for i in range(self.reps):
                #apply the ansatz
                circuit = self.ansatz(circuit, storage, self.params[idx_start:idx_end])
                idx_start = idx_end
                idx_end += self.n_qubits
            
            
            circuit = self.ansatz_bias(circuit, storage, self.params[-1])
            circuit.measure(storage[-1], clas_reg)
            
            job = qk.execute(circuit, backend=qk.Aer.get_backend(
                'qasm_simulator'), shots=self.shots)
            
            counts = job.result().get_counts(circuit)
            
            output = 0
            for bitstring, samples in counts.items():
                if bitstring == "1":
                    output += samples

            output = output / self.shots
            
            outputs.append(output)
        
        return np.array(outputs).reshape(-1,1)
    
    def gradient(self, data, y_pred, y):
        eps = 1e-8 #add small value to derivative of cross entropy to avoid division by zero
        n_samples = data.shape[0]
        
        #cost_deriv = (y_pred - y) #mse derivative
        cost_deriv = (y_pred - y)/(y_pred*(1-y_pred) + eps) #cross entropy derivative
        partial_grad = np.zeros((self.n_params, data.shape[0]))
        
        for i in range(self.n_params):
            #parameter shift rule
            self.params[i] += np.pi/2
            shift_plus = self.evaluate(data)
            
            self.params[i] -= np.pi
            shift_minus = self.evaluate(data)
            
            self.params[i] += np.pi/2
            
            partial_grad[i] = 0.5*(shift_plus - shift_minus).flatten()
            
        
        gradients = 1/n_samples * partial_grad@cost_deriv #chain rule
        
        return gradients.flatten()
    
    def train(self, data, y, epochs=100, lr = 1):
        eps = 1e-8
        for i in tqdm(range(epochs)):
            y_pred = self.evaluate(data)
            gradient = self.gradient(data, y_pred, y)
            self.params = self.params - lr*gradient #update parameters
            
            print(f"CE: {-np.mean(y*np.log(y_pred + eps) + (1-y)*np.log(1 - y_pred + eps)):.4f}, accuracy: {np.mean(np.round(y_pred).astype(int) == y):.4f}")
      

# Iris Data

## Data Preparation

In [3]:
iris = load_iris()

### Only first two classes

In [4]:
x = iris.data
y = iris.target

x = x[y != 2]
y = y[y != 2].reshape(-1,1)
y = scaler(y, a=0, b=1)

print(x.shape, y.shape)

(100, 4) (100, 1)


### Normalize Input Data

In [5]:
x = scaler(x, a=0, b=np.pi)

## Train Model

In [6]:
np.random.seed(44)
model1 = QuantumKernel(n_qubits = 4, reps = 2, shots=1000)

In [7]:
model1.train(x, y, epochs = 20, lr=0.1)

  0%|          | 0/20 [00:00<?, ?it/s]

CE: 1.1399, accuracy: 0.0300
CE: 1.0215, accuracy: 0.0500
CE: 0.9096, accuracy: 0.0800
CE: 0.8092, accuracy: 0.1500
CE: 0.7211, accuracy: 0.3000
CE: 0.6474, accuracy: 0.6200
CE: 0.5820, accuracy: 0.8600
CE: 0.5269, accuracy: 0.9400
CE: 0.4753, accuracy: 0.9200
CE: 0.4319, accuracy: 0.9600
CE: 0.3970, accuracy: 0.9500
CE: 0.3595, accuracy: 0.9700
CE: 0.3342, accuracy: 0.9700
CE: 0.3122, accuracy: 0.9600
CE: 0.2883, accuracy: 0.9700
CE: 0.2686, accuracy: 0.9700
CE: 0.2526, accuracy: 0.9700
CE: 0.2368, accuracy: 0.9600
CE: 0.2273, accuracy: 0.9700
CE: 0.2130, accuracy: 0.9800


### Breast Cancer Data

In [8]:
data = load_breast_cancer()
x = data.data
y = data.target.reshape(-1, 1)

np.random.seed(42)
x, _, y, _ = train_test_split(x, y, train_size=100)
x = scaler(x[:,:4], a=0, b=np.pi) #4 first feature of the breast cancer data set(not the best to pick?)

In [9]:
np.random.seed(42)
model2 = QuantumKernel(n_qubits = 4, reps = 2, shots=1000)

In [10]:
model2.train(x, y, epochs = 20, lr=0.1)

  0%|          | 0/20 [00:00<?, ?it/s]

CE: 0.8067, accuracy: 0.3200
CE: 0.7824, accuracy: 0.3300
CE: 0.7603, accuracy: 0.3400
CE: 0.7527, accuracy: 0.3400
CE: 0.7254, accuracy: 0.4200
CE: 0.7134, accuracy: 0.4500
CE: 0.6925, accuracy: 0.4900
CE: 0.6787, accuracy: 0.5300
CE: 0.6597, accuracy: 0.5800
CE: 0.6469, accuracy: 0.6200
CE: 0.6357, accuracy: 0.6800
CE: 0.6244, accuracy: 0.7100
CE: 0.6100, accuracy: 0.7400
CE: 0.6016, accuracy: 0.7500
CE: 0.5942, accuracy: 0.7900
CE: 0.5826, accuracy: 0.7800
CE: 0.5753, accuracy: 0.7900
CE: 0.5702, accuracy: 0.8100
CE: 0.5641, accuracy: 0.8000
CE: 0.5568, accuracy: 0.8200


Can we change the encoder/ansatz to get higher accuracy?